# §1: Association Rule Analysis


### Purpose and Motivation
In our first section, we will showcase the use of association rule analysis in a marketing management context. We will employ a publicly available dataset of car insurance cold calls to uncover successful outcome associations based on customer marketing interactions. As a marketing manager, you may encounter situations where you have an unstructured dataset and aim to optimize your workforce strategy and increase sales based on previous successful executions. By using association rule analysis, you can gain valuable insights, such as the fact that customers tend to purchase two items simultaneously, suggesting that it may be advantageous to place those items in the same area to boost overall sales. Our use of an ideal dataset for association rule analysis will allow us to optimize marketing strategies based on data-driven insights. To begin, we will import two commonly used external libraries, numpy and pandas, which we will alias as np and pd, respectively, to simplify the rest of the code. Before performing the statistical procedures, we will first need to import and preprocess the data.

  ### Data
  
  Source:	https://www.kaggle.com/kondla/carinsurance

In [279]:
#Import libraries
import numpy as np
import pandas as pd 

data = pd.read_csv("carinsurance_train.csv")

data.head()

,Id,Age,Job,Marital,Education,Default,Balance,HHInsurance,CarLoan,Communication,LastContactDay,LastContactMonth,NoOfContacts,DaysPassed,PrevAttempts,Outcome,CallStart,CallEnd,CarInsurance
0,1,32,management,single,tertiary,0,1218,1,0,telephone,28,jan,2,-1,0,NaN,13:45:20,13:46:30,0
1,2,32,blue-collar,married,primary,0,1156,1,0,NaN,26,may,5,-1,0,NaN,14:49:03,14:52:08,0
2,3,29,management,single,tertiary,0,637,1,0,cellular,3,jun,1,119,1,failure,16:30:24,16:36:04,1
3,4,25,student,single,primary,0,373,1,0,cellular,11,may,2,-1,0,NaN,12:06:43,12:20:22,1
4,5,30,management,married,tertiary,0,2694,0,0,cellular,3,jun,1,-1,0,NaN,14:35:44,14:38:56,0


In [280]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                4000 non-null   int64 
 1   Age               4000 non-null   int64 
 2   Job               3981 non-null   object
 3   Marital           4000 non-null   object
 4   Education         3831 non-null   object
 5   Default           4000 non-null   int64 
 6   Balance           4000 non-null   int64 
 7   HHInsurance       4000 non-null   int64 
 8   CarLoan           4000 non-null   int64 
 9   Communication     3098 non-null   object
 10  LastContactDay    4000 non-null   int64 
 11  LastContactMonth  4000 non-null   object
 12  NoOfContacts      4000 non-null   int64 
 13  DaysPassed        4000 non-null   int64 
 14  PrevAttempts      4000 non-null   int64 
 15  Outcome           958 non-null    object
 16  CallStart         4000 non-null   object
 17  CallEnd       

### Procedure (Step 1): Pre-Processing

In [281]:
# Drop attributes that are numeric and will be difficult to discretize

data = data.drop(["Id", "LastContactDay", "LastContactMonth", "CallStart", "CallEnd", "NoOfContacts", "DaysPassed", "PrevAttempts"], axis = 1)

In [276]:
# Discretize the numeric values I want to keep

data['Age']=pd.qcut(data['Age'],3 ) 

data['Balance']=pd.qcut(data['Balance'],3 )

data.head()

,Age,Job,Marital,Education,Default,Balance,HHInsurance,CarLoan,Communication,Outcome,CarInsurance
0,"(17.999, 34.0]",management,single,tertiary,0,"(1075.0, 98417.0]",1,0,telephone,NaN,0
1,"(17.999, 34.0]",blue-collar,married,primary,0,"(1075.0, 98417.0]",1,0,NaN,NaN,0
2,"(17.999, 34.0]",management,single,tertiary,0,"(228.0, 1075.0]",1,0,cellular,failure,1
3,"(17.999, 34.0]",student,single,primary,0,"(228.0, 1075.0]",1,0,cellular,NaN,1
4,"(17.999, 34.0]",management,married,tertiary,0,"(1075.0, 98417.0]",0,0,cellular,NaN,0


In [277]:
data.isnull().sum()

Age                 0
Job                19
Marital             0
Education         169
Default             0
Balance             0
HHInsurance         0
CarLoan             0
Communication     902
Outcome          3042
CarInsurance        0
dtype: int64

In [259]:
data.replace(np.nan, 'unknown')

,Age,Job,Marital,Education,Default,Balance,HHInsurance,CarLoan,Communication,Outcome,CarInsurance
0,"(17.999, 34.0]",management,single,tertiary,0,"(1075.0, 98417.0]",1,0,telephone,unknown,0
1,"(17.999, 34.0]",blue-collar,married,primary,0,"(1075.0, 98417.0]",1,0,unknown,unknown,0
2,"(17.999, 34.0]",management,single,tertiary,0,"(228.0, 1075.0]",1,0,cellular,failure,1
3,"(17.999, 34.0]",student,single,primary,0,"(228.0, 1075.0]",1,0,cellular,unknown,1
4,"(17.999, 34.0]",management,married,tertiary,0,"(1075.0, 98417.0]",0,0,cellular,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...
3995,"(17.999, 34.0]",technician,single,tertiary,0,"(-3058.001, 228.0]",1,0,cellular,failure,1
3996,"(45.0, 95.0]",admin.,divorced,secondary,0,"(-3058.001, 228.0]",1,1,cellular,unknown,0
3997,"(17.999, 34.0]",admin.,single,secondary,0,"(-3058.001, 228.0]",0,1,cellular,unknown,0
3998,"(34.0, 45.0]",entrepreneur,single,tertiary,0,"(228.0, 1075.0]",1,0,cellular,failure,0


In [260]:
data['Age'].unique()

[(17.999, 34.0], (34.0, 45.0], (45.0, 95.0]]
Categories (3, interval[float64]): [(17.999, 34.0] < (34.0, 45.0] < (45.0, 95.0]]

In [261]:
data['Balance'].unique()

[(1075.0, 98417.0], (228.0, 1075.0], (-3058.001, 228.0]]
Categories (3, interval[float64]): [(-3058.001, 228.0] < (228.0, 1075.0] < (1075.0, 98417.0]]

In [262]:
data.to_csv('Buyers_Discrete.csv', sep='\t',index=None)

In [263]:
file1=open('Buyers_Discrete.csv','r')

transactions01 = [] 

line=file1.readline() #header
h=line.strip().split("\t")
#print(h)
no_columns=len(h)

line=file1.readline() #first line of data

while line:
    values=line.strip().split("\t")         #split by tab \t
    has_item = {}
    for i in range(no_columns):                     #for each of columns
        item=h[i]+'='+values[i]           
        has_item[item] = 1
    transactions01.append(has_item)   
    line=file1.readline()

file1.close()

data = pd.DataFrame(transactions01)

data.fillna(0, inplace = True)

data.to_csv('Buyers-Items.csv', sep='\t',index=None)

data.head()

,"Age=(17.999, 34.0]",Job=management,Marital=single,Education=tertiary,Default=0,"Balance=(1075.0, 98417.0]",HHInsurance=1,CarLoan=0,Communication=telephone,Outcome=,...,Default=1,Job=self-employed,Education=,Outcome=other,Job=retired,Outcome=success,Job=,Job=housemaid,Job=entrepreneur,Job=unemployed
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Procedure (Step 2): Generate Frequency Itemsets
Generating frequency itemsets is a key step in association rule analysis. The statistical procedure involves identifying sets of items that occur together in a dataset with a frequency that exceeds a given threshold, known as the support threshold. Once frequent itemsets have been identified, association rule analysis can be used to explore the relationships between the items in the dataset.

- By FP-Growth

In [264]:
from mlxtend.frequent_patterns import fpgrowth
freq_itemsets = fpgrowth(data, min_support = 0.05, use_colnames = True)
freq_itemsets.to_csv('Final_Buyers-Items.csv', sep='\t',index=None)
freq_itemsets.tail()

,support,itemsets
4359,0.05400,"(Default=0, Job=retired, HHInsurance=0)"
4360,0.05375,"(Job=retired, HHInsurance=0, Age=(45.0, 95.0])"
4361,0.05025,"(Job=retired, HHInsurance=0, CarLoan=0)"
4362,0.05325,"(Default=0, Job=retired, HHInsurance=0, Age=(4..."
4363,0.05025,"(Default=0, Job=retired, HHInsurance=0, CarLoa..."


### Procedure (Step 4): Generate Association Rules
Association rules are generated by examining the co-occurrence of items within frequent itemsets and calculating measures of association, such as confidence and lift.

In [265]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(freq_itemsets, metric = "lift", min_threshold = 2)
rules.to_csv('Final_Buyers-Rules.csv', sep='\t',index=None)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Communication=cellular, Age=(17.999, 34.0])",(Marital=single),0.25925,0.30325,0.15725,0.606557,2.000189,0.078632,1.770906
1,"(Communication=cellular, Marital=single)","(Age=(17.999, 34.0])",0.22650,0.34100,0.15725,0.694260,2.035955,0.080013,2.155430
2,"(Age=(17.999, 34.0])","(Communication=cellular, Marital=single)",0.34100,0.22650,0.15725,0.461144,2.035955,0.080013,1.435448
3,(Marital=single),"(Communication=cellular, Age=(17.999, 34.0])",0.30325,0.25925,0.15725,0.518549,2.000189,0.078632,1.538578
4,"(Communication=cellular, Outcome=, Marital=sin...","(Age=(17.999, 34.0])",0.15000,0.34100,0.10500,0.700000,2.052786,0.053850,2.196667


In [266]:
# Filter association rules based on successful outcomes 

rules_filtered=rules[rules['consequents']=={'Outcome=success'}]     #Can specify condition to filter rules
rules_filtered.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3759,(CarInsurance=1),(Outcome=success),0.40100,0.0815,0.07275,0.181421,2.226030,0.040068,1.122067
3761,"(Default=0, CarInsurance=1)",(Outcome=success),0.39750,0.0815,0.07275,0.183019,2.245630,0.040354,1.124261
3765,"(CarLoan=0, CarInsurance=1)",(Outcome=success),0.36300,0.0815,0.07025,0.193526,2.374554,0.040666,1.138909
3769,"(Communication=cellular, CarInsurance=1)",(Outcome=success),0.32825,0.0815,0.06600,0.201066,2.467071,0.039248,1.149657
3773,"(Default=0, CarLoan=0, CarInsurance=1)",(Outcome=success),0.36100,0.0815,0.07025,0.194598,2.387710,0.040829,1.140425


In [268]:
rules_filtered.sort_values('lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3856,"(Default=0, Communication=cellular, HHInsuranc...",(Outcome=success),0.21375,0.0815,0.05000,0.233918,2.870161,0.032579,1.198958
3849,"(Communication=cellular, HHInsurance=0, CarIns...",(Outcome=success),0.21500,0.0815,0.05000,0.232558,2.853474,0.032478,1.196833
3832,"(Default=0, HHInsurance=0, CarLoan=0, CarInsur...",(Outcome=success),0.23725,0.0815,0.05475,0.230769,2.831524,0.035414,1.194050
3825,"(HHInsurance=0, CarLoan=0, CarInsurance=1)",(Outcome=success),0.23850,0.0815,0.05475,0.229560,2.816684,0.035312,1.192176
3817,"(Default=0, HHInsurance=0, CarInsurance=1)",(Outcome=success),0.25150,0.0815,0.05600,0.222664,2.732074,0.035503,1.181600
3813,"(HHInsurance=0, CarInsurance=1)",(Outcome=success),0.25325,0.0815,0.05600,0.221125,2.713195,0.035360,1.179266
3796,"(Default=0, Communication=cellular, CarLoan=0,...",(Outcome=success),0.29700,0.0815,0.06400,0.215488,2.644027,0.039794,1.170792
3789,"(Communication=cellular, CarLoan=0, CarInsuran...",(Outcome=success),0.29850,0.0815,0.06400,0.214405,2.630741,0.039672,1.169178
3781,"(Default=0, Communication=cellular, CarInsuran...",(Outcome=success),0.32525,0.0815,0.06600,0.202921,2.489826,0.039492,1.152332
3769,"(Communication=cellular, CarInsurance=1)",(Outcome=success),0.32825,0.0815,0.06600,0.201066,2.467071,0.039248,1.149657


## Results & Discussion

Association rule analysis has played a crucial role in discovering the rules that increase a bank customer's likelihood of purchasing insurance, resulting in a potential marketing performance improvement ranging from approximately 120%-180% increase in success rate. Most of the rules identified by association rule analysis suggest that customers who buy insurance are existing insurance subscribers, highlighting the importance of customer loyalty. Moreover, the overall trend observed through association rule analysis is that potential customers tend to possess attributes reflecting "responsible" decision-making, such as having car and home owners insurance, and not having a car loan or credit in default. Importantly, association rule analysis has shown that cellular marketing tends to be the most successful method of reaching out to such potential customers.